In [1]:
import pandas as pd
import numpy as np
import requests
import csv
import json
from collections import OrderedDict
import scipy.stats

### Collect and filter dataset for useful values and assign column names

In [2]:
language_data = pd.read_csv("DDW-C18-0000.csv")

In [3]:
language_data.head(7)

,"C-18 POPULATION BY BILINGUALISM, TRILINGUALISM, AGE AND SEX",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,State,District,Area Name,Total/,Age-group,Number speaking second language,NaN,NaN,Number speaking third language,NaN,NaN
1,code,code,NaN,Rural/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,Urban,NaN,Persons,Males,Females,Persons,Males,Females
3,1,2,3,4,5,6,7,8,9,10,11
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957


In [4]:
language_data = language_data.loc[5:,:]

In [5]:
['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3']

['State_Code',
 'District_Code',
 'Area_Name',
 'Total/Rural/Urban',
 'Age-group',
 'Number_speaking_second_language',
 'Males2',
 'Females2',
 'Number_speaking_third_language',
 'Males3',
 'Females3']

In [6]:
language_data_total = language_data.set_axis(['State_Code','District_Code','Area_Name','Total/Rural/Urban','Age-group','Number_speaking_second_language','Males2','Females2','Number_speaking_third_language','Males3','Females3'], axis=1, inplace=False)
language_data_total

,State_Code,District_Code,Area_Name,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
5,00,00,INDIA,Total,Total,314988770,176696383,138292387,86009580,50536832,35472748
6,00,00,INDIA,Total,5-9,15649192,8166843,7482349,1844108,978151,865957
7,00,00,INDIA,Total,10-14,34488492,18133423,16355069,7254335,3831131,3423204
8,00,00,INDIA,Total,15-19,42424599,22750908,19673691,12626717,6792766,5833951
9,00,00,INDIA,Total,20-24,41344406,22386694,18957712,12834334,7067614,5766720
...,...,...,...,...,...,...,...,...,...,...,...
1080,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,25-29,13065,7012,6053,4364,2355,2009
1081,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,30-49,38406,21301,17105,13320,7824,5496
1082,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,50-69,12899,7903,4996,4177,2842,1335
1083,35,00,ANDAMAN & NICOBAR ISLANDS,Urban,70+,1644,1016,628,432,305,127


In [7]:
indexNames = language_data_total[language_data_total['Area_Name'] == 'INDIA' ].index
language_data = language_data_total.drop(indexNames , inplace=False)

In [8]:
data = pd.read_csv("census_data.csv")
data

,State,District,Subdistt,Town/Village,Ward,EB,Level,Name,TRU,No_HH,...,MARG_AL_0_3_F,MARG_HH_0_3_P,MARG_HH_0_3_M,MARG_HH_0_3_F,MARG_OT_0_3_P,MARG_OT_0_3_M,MARG_OT_0_3_F,NON_WORK_P,NON_WORK_M,NON_WORK_F
0,0,0,0,0,0,0,India,India,Total,249501663,...,5820403,1420348,491424,928924,5518450,3082613,2435837,728966109,291330383,437635726
1,0,0,0,0,0,0,India,India,Rural,168612897,...,5640822,1154975,386077,768898,3803400,1972348,1831052,485005760,200944045,284061715
2,0,0,0,0,0,0,India,India,Urban,80888766,...,179581,265373,105347,160026,1715050,1110265,604785,243960349,90386338,153574011
3,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Total,2119718,...,44570,26585,6780,19805,108457,56880,51577,8218589,3445572,4773017
4,1,0,0,0,0,0,STATE,JAMMU & KASHMIR,Rural,1553433,...,40936,21963,5757,16206,89232,45625,43607,5994979,2562471,3432508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Rural,25475,...,119,30,21,9,1047,654,393,65311,23633,41678
2024,35,639,0,0,0,0,DISTRICT,North & Middle Andaman,Urban,724,...,1,0,0,0,70,43,27,1707,616,1091
2025,35,640,0,0,0,0,DISTRICT,South Andaman,Total,59064,...,63,48,24,24,1311,933,378,141311,49720,91591
2026,35,640,0,0,0,0,DISTRICT,South Andaman,Rural,23767,...,62,42,20,22,777,550,227,57596,19971,37625


In [9]:
data = data[["Level","Name","TRU","TOT_P","TOT_M","TOT_F"]]

In [10]:
df = data[data['Level'] == "STATE"]

In [11]:
census_dataset = df[df['TRU'] == "Total"]
census_dataset

,Level,Name,TRU,TOT_P,TOT_M,TOT_F
3,STATE,JAMMU & KASHMIR,Total,12541302,6640662,5900640
72,STATE,HIMACHAL PRADESH,Total,6864602,3481873,3382729
111,STATE,PUNJAB,Total,27743338,14639465,13103873
174,STATE,CHANDIGARH,Total,1055450,580663,474787
180,STATE,UTTARAKHAND,Total,10086292,5137773,4948519
222,STATE,HARYANA,Total,25351462,13494734,11856728
288,STATE,NCT OF DELHI,Total,16787941,8987326,7800615
318,STATE,RAJASTHAN,Total,68548437,35550997,32997440
420,STATE,UTTAR PRADESH,Total,199812341,104480510,95331831
636,STATE,BIHAR,Total,104099452,54278157,49821295


### Calculating State Level Data

In [12]:
df1 = language_data.groupby(['Area_Name'])
df1.first()

,State_Code,District_Code,Total/Rural/Urban,Age-group,Number_speaking_second_language,Males2,Females2,Number_speaking_third_language,Males3,Females3
Area_Name,,,,,,,,,,
ANDAMAN & NICOBAR ISLANDS,35,00,Total,Total,257425,142378,115047,72708,41846,30862
ANDHRA PRADESH,28,00,Total,Total,21583334,12193992,9389342,6344376,3836382,2507994
ARUNACHAL PRADESH,12,00,Total,Total,885945,478413,407532,418551,236666,181885
ASSAM,18,00,Total,Total,14460591,7979404,6481187,4352497,2636287,1716210
BIHAR,10,00,Total,Total,16761785,9443495,7318290,1682572,1028475,654097
CHANDIGARH,04,00,Total,Total,579920,322898,257022,321979,178557,143422
CHHATTISGARH,22,00,Total,Total,3384223,1820621,1563602,422569,245150,177419
DADRA & NAGAR HAVELI,26,00,Total,Total,198983,113211,85772,71516,44256,27260
DAMAN & DIU,25,00,Total,Total,116629,71260,45369,34602,21856,12746


In [13]:
group_state = list(df1.groups.keys())
group_state

['ANDAMAN & NICOBAR ISLANDS',
 'ANDHRA PRADESH',
 'ARUNACHAL PRADESH',
 'ASSAM',
 'BIHAR',
 'CHANDIGARH',
 'CHHATTISGARH',
 'DADRA & NAGAR HAVELI',
 'DAMAN & DIU',
 'GOA',
 'GUJARAT',
 'HARYANA',
 'HIMACHAL PRADESH',
 'JAMMU & KASHMIR',
 'JHARKHAND',
 'KARNATAKA',
 'KERALA',
 'LAKSHADWEEP',
 'MADHYA PRADESH',
 'MAHARASHTRA',
 'MANIPUR',
 'MEGHALAYA',
 'MIZORAM',
 'NAGALAND',
 'NCT OF DELHI',
 'ODISHA',
 'PUDUCHERRY',
 'PUNJAB',
 'RAJASTHAN',
 'SIKKIM',
 'TAMIL NADU',
 'TRIPURA',
 'UTTAR PRADESH',
 'UTTARAKHAND',
 'WEST BENGAL']

In [14]:
State_dict = dict()

In [15]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z.iloc[0]
    
    female_three_lang = int(z['Females3'])
    male_three_lang = int(z['Males3'])
    
    State_Code = z['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    population = population_row['TOT_P'].tolist()[0]
    female_population = population_row['TOT_F'].tolist()[0]
    male_population = population_row['TOT_M'].tolist()[0]
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("female_population = ",female_population)
    #print("male_population = ",male_population)
    
    male_prop = male_three_lang/male_population
    female_prop = female_three_lang/female_population
    pool_prop = (male_three_lang+female_three_lang)/population # Calculate Pool Proportion
    #print("male_prop = ",male_prop)
    #print("female_prop = ",female_prop)
    #print("pool_prop = ",pool_prop)
    
    male_percent = (male_three_lang*100)/male_population
    female_percent = (female_three_lang*100)/female_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/male_population) + ((pool_prop*(1-pool_prop))/female_population) ) # Calculate Standard Error
    point_estimate = male_prop-female_prop # Calculate Point Estimate
    #print("point_estimate = ",point_estimate)
    null_value = 0                         # Null value for the Null Hypothesis
    z = (point_estimate - null_value)/SE   # Calculate z statistic
    p_value = scipy.stats.norm.sf(abs(z))*2  # Calculate p value for two tailed statistic
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("male_percent = ",male_percent)
    #print("female_percent = ",female_percent)
    #print("P value = ",p_value)
    
    State_dict[State_Code] = [round(male_percent,3),round(female_percent,4),p_value]
    
    #print("")
    #print("")

In [16]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [18.955, 14.1928, 0.0]),
             ('02', [5.758, 4.3399, 0.0]),
             ('03', [29.945, 26.2981, 0.0]),
             ('04', [30.751, 30.2077, 1.6798262656386235e-09]),
             ('05', [2.132, 1.5822, 0.0]),
             ('06', [5.048, 4.1271, 0.0]),
             ('07', [8.251, 7.906, 4.6577229244531145e-147]),
             ('08', [1.808, 1.1028, 0.0]),
             ('09', [1.481, 1.0895, 0.0]),
             ('10', [1.895, 1.3129, 0.0]),
             ('11', [30.776, 28.5123, 3.820070243700941e-83]),
             ('12', [33.151, 27.1545, 0.0]),
             ('13', [29.922, 24.2058, 0.0]),
             ('14', [24.764, 18.3864, 0.0]),
             ('15', [6.251, 3.3684, 0.0]),
             ('16', [6.89, 3.8584, 0.0]),
             ('17', [10.775, 7.7409, 0.0]),
             ('18', [16.539, 11.2419, 0.0]),
             ('19', [3.823, 2.455, 0.0]),
             ('20', [5.706, 3.9051, 0.0]),
             ('21', [15.836, 10.4328, 0.0]),
             ('22', [1.9

In [17]:
a_file = open("gender-india-c.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'male-percentage', 'female-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [18]:
data1 = pd.read_csv("gender-india-c.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Three or More Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Three or More Langage)

 state-code  male-percentage  female-percentage       p-value
          1           18.955            14.1928  0.000000e+00
          2            5.758             4.3399  0.000000e+00
          3           29.945            26.2981  0.000000e+00
          4           30.751            30.2077  1.679826e-09
          5            2.132             1.5822  0.000000e+00
          6            5.048             4.1271  0.000000e+00
          7            8.251             7.9060 4.657723e-147
          8            1.808             1.1028  0.000000e+00
          9            1.481             1.0895  0.000000e+00
         10            1.895             1.3129  0.000000e+00
         11           30.776            28.5123  3.820070e-83
         12           33.151            27.1545  0.000000e+00
         13           29.922            24.2058  0.000000e+00
         14           24.764            18.3864  0.00

### Exactly 2 Langage

In [19]:
State_dict = dict()

In [20]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z.iloc[0]
    
    female_three_lang = int(z['Females3'])
    male_three_lang = int(z['Males3'])
    
    female_two_lang = int(z['Females2'])
    male_two_lang = int(z['Males2'])
    
    female_exactly_two = female_two_lang - female_three_lang
    male_exactly_two = male_two_lang - male_three_lang
    
    
    State_Code = z['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    population = population_row['TOT_P'].tolist()[0]
    female_population = population_row['TOT_F'].tolist()[0]
    male_population = population_row['TOT_M'].tolist()[0]
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("female_population = ",female_population)
    #print("male_population = ",male_population)
    
    male_prop = male_exactly_two/male_population
    female_prop = female_exactly_two/female_population
    pool_prop = (male_exactly_two+female_exactly_two)/population
    #print("male_prop = ",male_prop)
    #print("female_prop = ",female_prop)
    #print("pool_prop = ",pool_prop)
    
    male_percent = (male_exactly_two*100)/male_population
    female_percent = (female_exactly_two*100)/female_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/male_population) + ((pool_prop*(1-pool_prop))/female_population) )
    point_estimate = male_prop-female_prop
    #print("point_estimate = ",point_estimate)
    null_value = 0
    z = (point_estimate - null_value)/SE
    p_value = scipy.stats.norm.sf(abs(z))*2
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("male_percent = ",male_percent)
    #print("female_percent = ",female_percent)
    #print("P value = ",p_value)
    
    State_dict[State_Code] = [round(male_percent,3),round(female_percent,4),p_value]
    
    #print("")
    #print("")

In [21]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [35.781, 28.9953, 0.0]),
             ('02', [13.886, 12.1775, 0.0]),
             ('03', [19.812, 17.59, 0.0]),
             ('04', [24.858, 23.9265, 1.604889370845333e-28]),
             ('05', [16.996, 14.079, 0.0]),
             ('06', [19.092, 15.8591, 0.0]),
             ('07', [33.947, 30.8698, 0.0]),
             ('08', [10.54, 8.2411, 0.0]),
             ('09', [11.258, 8.9425, 0.0]),
             ('10', [15.504, 13.3762, 0.0]),
             ('11', [35.26, 32.5849, 1.7889561419295985e-107]),
             ('12', [33.862, 33.688, 0.03023856552480571]),
             ('13', [34.676, 35.3237, 1.3400907589589068e-21]),
             ('14', [28.086, 25.9496, 0.0]),
             ('15', [25.111, 21.645, 0.0]),
             ('16', [31.176, 26.9884, 0.0]),
             ('17', [18.791, 18.1049, 2.107428886980247e-52]),
             ('18', [33.521, 31.2127, 0.0]),
             ('19', [14.939, 11.5858, 0.0]),
             ('20', [26.628, 24.2837, 0.0]),
             ('21'

In [22]:
a_file = open("gender-india-b.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'male-percentage', 'female-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [23]:
print("")

In [24]:
data1 = pd.read_csv("gender-india-b.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Exactly two Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Exactly two Langage)

 state-code  male-percentage  female-percentage       p-value
          1           35.781            28.9953  0.000000e+00
          2           13.886            12.1775  0.000000e+00
          3           19.812            17.5900  0.000000e+00
          4           24.858            23.9265  1.604889e-28
          5           16.996            14.0790  0.000000e+00
          6           19.092            15.8591  0.000000e+00
          7           33.947            30.8698  0.000000e+00
          8           10.540             8.2411  0.000000e+00
          9           11.258             8.9425  0.000000e+00
         10           15.504            13.3762  0.000000e+00
         11           35.260            32.5849 1.788956e-107
         12           33.862            33.6880  3.023857e-02
         13           34.676            35.3237  1.340091e-21
         14           28.086            25.9496  0.0000

### Exactly 1 Langage

In [25]:
State_dict = dict()

In [26]:
for i in range(len(group_state)):
    key = group_state[i]
    z = df1.get_group(group_state[i])          #retrieve the dataset under that state
    z = z.iloc[0]
    
    
    female_two_lang = int(z['Females2'])
    male_two_lang = int(z['Males2'])
       
    State_Code = z['State_Code']
    State_Name = key
    population_row = census_dataset[census_dataset["Name"] == State_Name]
    population = population_row['TOT_P'].tolist()[0]
    female_population = population_row['TOT_F'].tolist()[0]
    male_population = population_row['TOT_M'].tolist()[0]
    
    female_exactly_one = female_population - female_two_lang
    male_exactly_one = male_population - male_two_lang
    
    #print("State_Name = ",State_Name)
    #print("State_Code = ",State_Code)
    #print("population = ",population)
    #print("female_population = ",female_population)
    #print("male_population = ",male_population)
    
    male_prop = male_exactly_one/male_population
    female_prop = female_exactly_one/female_population
    pool_prop = (male_exactly_one+female_exactly_one)/population
    #print("male_prop = ",male_prop)
    #print("female_prop = ",female_prop)
    #print("pool_prop = ",pool_prop)
    
    male_percent = (male_exactly_one*100)/male_population
    female_percent = (female_exactly_one*100)/female_population
    SE = np.sqrt( ((pool_prop*(1-pool_prop))/male_population) + ((pool_prop*(1-pool_prop))/female_population) )
    point_estimate = male_prop-female_prop
    #print("point_estimate = ",point_estimate)
    null_value = 0
    z = (point_estimate - null_value)/SE
    p_value = scipy.stats.norm.sf(abs(z))*2
    #print("SE = ",SE)
    #print("z = ",z)
    
    
    #print("")
    #print("male_percent = ",male_percent)
    #print("female_percent = ",female_percent)
    #print("P value = ",p_value)
    
    State_dict[State_Code] = [round(male_percent,3),round(female_percent,4),p_value]
    
    #print("")
    #print("")

In [27]:
State_dict = OrderedDict(sorted(State_dict.items()))
State_dict

OrderedDict([('01', [45.264, 56.812, 0.0]),
             ('02', [80.356, 83.4826, 0.0]),
             ('03', [50.243, 56.1119, 0.0]),
             ('04', [44.391, 45.8658, 8.246713581904903e-52]),
             ('05', [80.872, 84.3389, 0.0]),
             ('06', [75.86, 80.0137, 0.0]),
             ('07', [57.802, 61.2241, 0.0]),
             ('08', [87.652, 90.656, 0.0]),
             ('09', [87.261, 89.968, 0.0]),
             ('10', [82.602, 85.3109, 0.0]),
             ('11', [33.965, 38.9027, 0.0]),
             ('12', [32.987, 39.1575, 0.0]),
             ('13', [35.402, 40.4705, 0.0]),
             ('14', [47.15, 55.664, 0.0]),
             ('15', [68.638, 74.9867, 0.0]),
             ('16', [61.934, 69.1531, 0.0]),
             ('17', [70.434, 74.1542, 0.0]),
             ('18', [49.939, 57.5453, 0.0]),
             ('19', [81.238, 85.9593, 0.0]),
             ('20', [67.666, 71.8112, 0.0]),
             ('21', [62.919, 71.2966, 0.0]),
             ('22', [85.813, 87.7001, 0.0])

In [28]:
a_file = open("gender-india-a.csv", "w",newline="")
writer = csv.DictWriter(a_file, fieldnames = ['state-code', 'male-percentage', 'female-percentage', 'p-value'])
writer.writeheader()
writer = csv.writer(a_file)
for key, value in State_dict.items():
    writer.writerow([key] + value)

a_file.close()

In [29]:
print("")

In [30]:
data1 = pd.read_csv("gender-india-a.csv")
data1 = data1.to_string(index=False)
print("Displaying states and union territories of India data (Exactly one Langage)")
print("")
print(data1)

Displaying states and union territories of India data (Exactly one Langage)

 state-code  male-percentage  female-percentage       p-value
          1           45.264            56.8120  0.000000e+00
          2           80.356            83.4826  0.000000e+00
          3           50.243            56.1119  0.000000e+00
          4           44.391            45.8658  8.246714e-52
          5           80.872            84.3389  0.000000e+00
          6           75.860            80.0137  0.000000e+00
          7           57.802            61.2241  0.000000e+00
          8           87.652            90.6560  0.000000e+00
          9           87.261            89.9680  0.000000e+00
         10           82.602            85.3109  0.000000e+00
         11           33.965            38.9027  0.000000e+00
         12           32.987            39.1575  0.000000e+00
         13           35.402            40.4705  0.000000e+00
         14           47.150            55.6640  0.0000